In [1]:
from sys import path
path.append(".")

# Intro

This notebook is just for some simple tests over the "impossible to interpret" scripts provided by the authors. I will try especially the data related part, since it is very weird the way they decided to handle it.

In [38]:
from repeat_src.utils.models import get_backbone_model
from os.path import join as join_paths
from numpy import expand_dims, array
from pandas import read_csv
from keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from tqdm.notebook import tqdm
from typing import List
from glob import glob

def load_images(file_list: List[str], batch_size: int) -> array:
    if batch_size>len(file_list):
        batch_size=len(file_list)
        
    count: int = 0
    x: list = []
    for path in file_list:
        x_temp = image.load_img(path)
        x_temp = image.img_to_array(x_temp)
        x_temp = expand_dims(x_temp, axis=0)
        x_temp= preprocess_input(x_temp, version=2)
        
        count += 1
        x.append(x_temp)
        if count % batch_size == 0 and count != 0:
            x = array(x)
            x = x.reshape(batch_size, 256, 256, 3)
            yield x
            x = []
            

In [36]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [28]:
input_shape: tuple = (256, 256, 3)
batch_size: int = 16

In [4]:
model = get_backbone_model(input_shape=input_shape)

58925056/58909280 [==============================] - 2s 0us/step


In [21]:
# make a list of unique videos (join video and utterance from the table with ground truths)
validation_ground_truth_df = read_csv("../omg_ValidationVideos.csv")

video_id_list: list = validation_ground_truth_df.apply(
    lambda x: f"{x['video']}/{x['utterance'].split('.')[0]}", axis=1
).tolist()

paths_to_faces = [
    join_paths("/data/leonardo/OMGEmotionChallenge/Validation_Set/trimmed_faces/", video_id)
    for video_id in video_id_list
]



In [33]:
paths_to_faces = paths_to_faces[:3]

In [46]:
extracted_features: list = []
for current_video_path in tqdm(paths_to_faces, desc='Extracting features w/ VGG16-Face'):
    video_frames_paths = glob(join_paths(current_video_path, "*.png"))

    current_video_features = model.predict_generator(
        load_images(video_frames_paths, batch_size),
        (len(video_frames_paths) // batch_size) + 1,
        verbose=1,
    )
    extracted_features.append(current_video_features)
    # df_cnn=df_cnn[0:len(idict[k]),:]
    # df_cnn=pd.DataFrame(df_cnn)
    # df_cnn.to_csv(predict_path+k+'.csv',index=None)


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

# Checking feature extraction

In [1]:
import pandas as pd

In [7]:
example_features_extracted = pd.read_csv("./extracted_features/Validation_Set/0cf41d04d_1/utterance_28.csv")

In [8]:
example_features_extracted.values.shape

(304, 512)

The shape is `(frame_number, VGG_output_shape)`. However, the number of frame is slightly larger than the actual number, since it has to be a power of the batch size (in this case 16). I believe, when the number of frames is not enough, new frames are created from the last one.

# Audio feature extraction

In [1]:
audio_example_path: str = "/data/leonardo/OMGEmotionChallenge/Validation_Set/audio/0cf41d04d_1/utterance_28.wav"

In [2]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
y = smile.process_file(audio_example_path)

OSError: dlopen(/Users/leonardoalchieri/miniconda3/envs/omg-winner-code/lib/python3.8/site-packages/audresample/core/bin/libaudresample.dylib, 0x0006): tried: '/Users/leonardoalchieri/miniconda3/envs/omg-winner-code/lib/python3.8/site-packages/audresample/core/bin/libaudresample.dylib' (mach-o file, but is an incompatible architecture (have (x86_64), need (arm64e)))